In [1]:
import os

In [2]:
%pwd

'/Users/aksshar/PycharmProjects/End-to-end-Machine-Learning-Project-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/aksshar/PycharmProjects/End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/rmshakshar/MLOPS.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="rmshakshar"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6e629236b7cbf69bf62c1be489698092e77ba87e"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/rmshakshar/MLOPS.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import tensorflow
from tensorflow.keras.models import load_model


In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        print("load start")
        model = load_model(self.config.model_path)
        print("load complete")
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            print("predict")
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)


            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="Artificial neural network")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-09-19 21:49:24,944: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-19 21:49:24,948: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-19 21:49:24,950: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-19 21:49:24,951: INFO: common: created directory at: artifacts]
[2023-09-19 21:49:24,952: INFO: common: created directory at: artifacts/model_evaluation]
load start
load complete
predict
1368/1368 [==============================] - 1s 441us/step
shape:  [[8.6045256e-19 0.0000000e+00 3.4570044e-37 ... 2.6305123e-37
  0.0000000e+00 2.7162474e-32]
 [4.5749075e-27 4.2636823e-04 2.5659722e-16 ... 1.2402303e-18
  3.0032235e-12 3.1261023e-22]
 [6.5692754e-27 9.9111861e-01 9.3573380e-13 ... 7.9998356e-20
  1.9463632e-06 1.9035029e-18]
 ...
 [1.0890216e-11 2.4923912e-11 2.3531585e-12 ... 1.2084220e-05
  1.5504258e-16 2.0946501e-17]
 [2.2908369e-24 1.9142534e-04 3.5654874e-15 ... 2.6485121e-16
  3.7583478e-12 5.8389415e-21]
 [1.0000

2023/09/19 21:49:30 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/tx/7df2_9m92_74rf9f7px2j44r0000gn/T/tmpv1e9ajsd/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.3.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
/Users/aksshar/PycharmProjects/End-to-end-Machine-Learning-Project-with-MLflow/venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'Artificial neural network'.
2023/09/19 21:49:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Artificial neural network, version 1
Created version '1' of model 'Artificial neural network'.
